In [ ]:
import numpy as np
from fgn import fbm
from crossing_tree import *

In [ ]:
generator = fbm( N = 2**21+1, H = .65 )
generator.set_rnd( np.random.RandomState( 123 ) )
max_levels, max_crossings = 7, 20

In [ ]:
T, X = generator( )
# delta = np.std( np.diff( X ) )
delta = np.subtract( *np.percentile( np.diff( X ), [ 75, 25 ] ) )
Tnk, XTnk, Znk, Vnk, Wnk = xtree_build( T, X, delta = delta )

In [ ]:
# Nn = np.array( [ len( Tk ) - 1 for Tk in Tnk ],
#     dtype = np.float ).reshape( ( len( Tnk ), -1 ) )
Nn = np.zeros( ( max_levels + 1, 1 ), dtype = np.int )
for n, Tk in enumerate( Tnk, 0 ) :
    n = max_levels if n > max_levels else n
    Nn[ n ] += len( Tk ) - 1
Dnk = np.zeros( ( max_levels + 1, max_crossings // 2 ), dtype = np.int )
for n, Zk in enumerate( Znk[ 1: ], 0 ) :
    n = max_levels if n > max_levels else n
    Z_count, Z_freq = np.unique( Zk, return_counts = True )
    Z_count = np.minimum( Z_count, max_crossings )
    mask = ( Z_count < max_crossings )
    Dnk[ n, Z_count[ mask ] // 2 - 1 ] += Z_freq[ mask ]
    Dnk[ n, max_crossings // 2 - 1 ] += np.sum( Z_freq[ ~mask ] )
Vnde = np.zeros( ( max_levels + 1, 2, 2 ), dtype = np.int )
for n, Vk in enumerate( Vnk[ 1: ], 0 ) :
    n = max_levels if n > max_levels else n
    Vnde[ n, 0 ] += np.sum( Vk[ Vk[ :, 2 ] < 0 ], axis = 0 )[:2]
    Vnde[ n, 1 ] += np.sum( Vk[ Vk[ :, 2 ] > 0 ], axis = 0 )[:2]


In [ ]:
print Dnk[0,:]#.dot( np.arange( 2, max_crossings+2, 2 ) )
print Nn
print Znk[1]

In [ ]:
Vnd = np.sum( Vnde, axis = 2, dtype = np.float ).reshape( VDn.shape[:2] + ( 1, ) )
Dn = np.sum( Dnk, axis = 1, dtype = np.float ).reshape( Dnk.shape[:1] + ( 1, ) )

EXnde = Vnde / Vnd
CDnk = Dnk / Dn
print CDnk
print EXnde

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# dat = np.load( './output/fbm_iqr_20150429-072004_18_0.50_200.npz' )
# dat = np.load( './output/fbm_iqr_20150429-072224_18_0.72_200.npz' )
# dat = np.load( './output/fbm_iqr_20150429-072516_18_0.95_200.npz' )
# dat = np.load( './output/fbm_iqr_20150429-083530_16_0.7250_200.npz' )
# dat = np.load( './output/fbm_iqr_20150429-083459_16_0.5000_200.npz' )
# dat = np.load( './output/fbm_iqr_20150429-090509_21_0.5000_1000.npz' )
dat = np.load( './output/fbm_iqr_20150429-150719_21_0.6500_1000.npz' )

Njn = dat['Njn']
Djnk = dat['Djnk']
Vjnde = dat['Vjnde']

Djn = np.sum( Djnk, axis = 2, dtype = np.float ).reshape( Djnk.shape[:2] + ( 1, ) )
Vjnd = np.sum( Vjnde, axis = 3, dtype = np.float ).reshape( Vjnde.shape[:3] + ( 1, ) )

Phat_jnk = Djnk / Djn
Qhat_jnde = Vjnde / Vjnd

Phat_nk = Phat_jnk.mean( axis = 0, dtype = np.float64 )
Qhat_nde = Qhat_jnde.mean( axis = 0, dtype = np.float64 )

sdPhat_nk = Phat_jnk.std( axis = 0, dtype = np.float64 )
sdQhat_nde = Qhat_jnde.std( axis = 0, dtype = np.float64 )

In [ ]:
# plt.figure( figsize = ( 16, 9 ) )
# for Phat_k in Phat_nk[:5] :
#     plt.plot( np.log( Phat_k ) )
H = 0.65
theta = 2**( 1 - 1/H )
theoretical = [ theta*(1-theta)**k for k in range( 0, 10 ) ]
level = 5
plt.figure( figsize = ( 16, 9 ) )
plt.subplot(221)
plt.hist( Phat_jnk[:,level,0], color = 'gray' )
plt.title( "Simulated distribution: res. $\\delta 2^{%d}$" % level )
plt.xlabel( "$\\hat{\\mathbb{P}}(Z = 2)$" )
plt.axvline(theoretical[0], color='y', linewidth=2)
plt.axvline(Phat_nk[level,0], color='r', linewidth=2)
plt.axvline(Phat_nk[level,0] - sdPhat_nk[level,0], color='pink', linestyle='dashed', linewidth=2)
plt.axvline(Phat_nk[level,0] + sdPhat_nk[level,0], color='pink', linestyle='dashed', linewidth=2)

plt.subplot(222)
plt.hist( Phat_jnk[:,level,1], color = 'gray' )
plt.title( "Simulated distribution: res. $\\delta 2^{%d}$" % level )
plt.xlabel( "$\\hat{\\mathbb{P}}(Z = 4)$" )
plt.axvline(theoretical[1], color='y', linewidth=2)
plt.axvline(Phat_nk[level,1], color='r', linewidth=2)
plt.axvline(Phat_nk[level,1] - sdPhat_nk[level,1], color='pink', linestyle='dashed', linewidth=2)
plt.axvline(Phat_nk[level,1] + sdPhat_nk[level,1], color='pink', linestyle='dashed', linewidth=2)

plt.subplot(223)
plt.hist( Phat_jnk[:,level,2], color = 'gray' )
plt.title( "Simulated distribution: res. $\\delta 2^{%d}$" % level )
plt.xlabel( "$\\hat{\\mathbb{P}}(Z = 6)$" )
plt.axvline(theoretical[2], color='y', linewidth=2)
plt.axvline(Phat_nk[level,2], color='r', linewidth=2)
plt.axvline(Phat_nk[level,2] - sdPhat_nk[level,2], color='pink', linestyle='dashed', linewidth=2)
plt.axvline(Phat_nk[level,2] + sdPhat_nk[level,2], color='pink', linestyle='dashed', linewidth=2)

plt.subplot(224)
plt.hist( Phat_jnk[:,level,6], color = 'gray' )
plt.title( "Simulated distribution: res. $\\delta 2^{%d}$" % level )
plt.xlabel( "$\\hat{\\mathbb{P}}(Z = 14)$" )
plt.axvline(theoretical[6], color='y', linewidth=2)
plt.axvline(Phat_nk[level,6], color='r', linewidth=2)
plt.axvline(Phat_nk[level,6] - sdPhat_nk[level,6], color='pink', linestyle='dashed', linewidth=2)
plt.axvline(Phat_nk[level,6] + sdPhat_nk[level,6], color='pink', linestyle='dashed', linewidth=2)

In [ ]:
print Djnk.shape

In [ ]:
print Njn.mean( axis = 0, dtype = np.float64 )
print Njn.std( axis = 0, dtype = np.float64 )

print Phat_jnk.mean( axis = 0, dtype = np.float64 )
print Phat_jnk.std( axis = 0, dtype = np.float64 )

print Qhat_jnde.mean( axis = 0, dtype = np.float64 )
print Qhat_jnde.std( axis = 0, dtype = np.float64 )

In [ ]:
theta = 2**(1-1/0.50)
print theta
print 1 / np.sqrt( 2 / (1-theta) )

Here I confirm that my implementation is correct.

In [ ]:
# %lprun -f xtree_integer_crossings
# %timeit -n10
lht0, lhp0 = xtree_integer_crossings( T, Z )

In [ ]:
# %lprun -f f_get_w_int
# %timeit -n10
mht0, mhp0, _, _ = f_get_w_int( T, Z )

In [ ]:
print np.allclose( mht0, lht0 )
print np.allclose( mhp0, lhp0 )

The following section concerns the correctness of the FGN generator.

In [ ]:
import numpy as np
from numpy.fft import rfft, fft
rnd = np.random.RandomState( 123 )

N = 2**15 + 1
sigma = 1.0 ; H = 0.5

In [ ]:
R = np.arange( N, dtype = np.float64 )
R = sigma * sigma * .5 * (
      np.abs( R - 1 ) ** ( 2.0 * H )
    + np.abs( R + 1 ) ** ( 2.0 * H )
    - 2 * np.abs( R ) ** ( 2.0 * H ) )

In [ ]:
Z = np.real( rfft( np.append( R, R[::-1][1:-1] ) ) )
Z = np.sqrt( np.maximum( Z, 0.0 ) / ( 2 * N - 2 ) )
Z = np.append( Z, Z[::-1][1:-1] )

In [ ]:
W = rnd.randn( 2 * N - 2 ) + rnd.randn( 2 * N - 2 ) * 1j
W = fft( Z * W )

u,v = ( np.real( W[ :N ] ), np.imag( W[ :N ] ) )

In [ ]:
u,v